In [50]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn import preprocessing
import optuna

# Step 2: Load the data

Next, we'll load the training and test data.  

We set `index_col=0` in the code cell below to use the `id` column to index the DataFrame.  (*If you're not sure how this works, try temporarily removing `index_col=0` and see how it changes the result.*)

In [47]:
# Load the training data
train = pd.read_csv("../input/train10fold/train-folds (1).csv")
test = pd.read_csv("../input/30daysofml/test.csv")
submission_data = pd.read_csv("../input/30daysofmlsubmisison/sample_submission.csv")
# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,3
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,7
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,3
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,6
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,8


In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.4.1


In [48]:
# Separate target from features
features = [col for col in train.columns if col not in ('id', 'target', 'kfold')]
object_cols = [col for col in features if 'cat' in col]

ordinal_encoder = OrdinalEncoder()

xtest = test[features]
xtest = xtest.copy()
xtest[object_cols] = ordinal_encoder.fit_transform(xtest[object_cols])

In [49]:
xtest = pd.get_dummies(xtest, )

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,4.0,8.0,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,0.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,7.0,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,3.0,10.0,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,1.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,5.0,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [26]:
train = train.copy()
train[object_cols] = ordinal_encoder.fit_transform(train[object_cols])

In [28]:
from IPython.display import display

xtrain = train.sample(frac = 0.8, random_state = 0)
xvalid = train.drop(xtrain.index)
display(xtrain.head(4))

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
112692,187716,0.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,...,0.377831,0.898393,0.384762,0.269084,0.377731,0.180624,0.167760,0.866318,7.206383,0
19498,32528,0.0,1.0,0.0,2.0,1.0,1.0,0.0,4.0,2.0,...,0.286945,0.352001,0.383425,0.503357,0.228779,0.354888,0.168037,0.627185,8.946281,0
31689,52668,1.0,1.0,0.0,2.0,1.0,1.0,0.0,4.0,4.0,...,0.399333,0.584227,0.134076,0.120760,0.347790,0.339751,0.273321,0.858320,7.510238,0
231780,386481,1.0,1.0,1.0,0.0,1.0,1.0,0.0,4.0,4.0,...,0.414747,0.415336,0.415271,0.374219,0.211966,0.380021,0.627377,0.332294,7.882394,0


In [29]:
ytrain = xtrain['target']
yvalid = xvalid['target']
xtrain = xtrain[features]
xvalid = xvalid[features]

In [33]:

# Scale to [0, 1]
max_ = xtrain.max(axis=0)
min_ = xtrain.min(axis=0)
xtrain = (xtrain - min_) / (max_ - min_)
xvalid = (xvalid - min_) / (max_ - min_)

In [ ]:
#Model 4
# Load the training data
train = pd.read_csv("../input/train10fold/train-folds (1).csv")
test = pd.read_csv("../input/30daysofml/test.csv")
submission_data = pd.read_csv("../input/30daysofmlsubmisison/sample_submission.csv")

# Separate target from features
features = [col for col in train.columns if col not in ('id', 'target', 'kfold')]
# Separate object columns from features
object_cols = [col for col in features if 'cat' in col]
# Separate numerical columns from the features
numerical_cols = [col for col in train.columns if col.startswith('cont')]


oneHotEnc = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore')

xtest = test[features]
xtest = xtest.copy()
xtest_ohe = oneHotEnc.fit_transform(xtest[object_cols])
xtest_ohe = pd.DataFrame(xtest_ohe, columns=[f'ohe_{i}' for i
                                             in range(xtest_ohe.shape[1])])
xtest = pd.concat([xtest, xtest_ohe], axis = 1)
xtest = xtest.drop(object_cols, axis = 1)

#scaler = preprocessing.StandardScaler()
#xtest = scaler.fit_transform(xtest)

final_valid_preds = {}
final_test_preds = []
rmse_score = []
for fold in range(10):
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    
    valid_ids = xvalid.id.values.tolist() # to keep all the ids of the validation data
        
    ytrain = xtrain.target
    yvalid = xvalid.target
        
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    xtrain_ohe= oneHotEnc.fit_transform(xtrain[object_cols])
    xvalid_ohe = oneHotEnc.transform(xvalid[object_cols])
    
    xtrain_ohe = pd.DataFrame(xtrain_ohe, columns=[f'ohe_{i}' for i
                                             in range(xtrain_ohe.shape[1])])
    xvalid_ohe = pd.DataFrame(xvalid_ohe, columns=[f'ohe_{i}' for i
                                             in range(xvalid_ohe.shape[1])])
    
    xtrain = pd.concat([xtrain, xtrain_ohe], axis = 1)
    xvalid = pd.concat([xvalid, xvalid_ohe], axis = 1)
    
    xtrain = xtrain.drop(object_cols, axis = 1)
    xvalid = xvalid.drop(object_cols, axis = 1)
    
    best_params = {'learning_rate': 0.07853392035787837,
                     'colsample_bytree': 0.170759104940733,
                     'max_depth': 3,
                     'reg_lambda': 1.7549293092194938e-05,
                     'reg_alpha': 14.68267919457715,
                     'subsample': 0.8031450486786944,
                     'alpha': 30
                  }
    model = XGBRegressor(objective='reg:squarederror',
                         n_estimators=5000,
                         random_state=0,
                         **best_params,
                         #tree_method='gpu_hist',
                         #gpu_id=0, 
                         #predictor='gpu_predictor'
                        )

    model.fit(xtrain, ytrain, 
              early_stopping_rounds=300, 
              eval_set=[(xvalid, yvalid)], 
              verbose=1000)
    
    valid_preds = model.predict(xvalid)
    test_preds = model.predict(xtest)
    
    final_valid_preds.update(dict(zip(valid_ids, valid_preds)))
    final_test_preds.append(test_preds)

    rmse = mean_squared_error(yvalid, valid_preds, squared=False)
    rmse_score.append(rmse)
    print(fold, rmse)
    
print(np.mean(rmse_score), np.std(rmse_score))

final_valid_preds = pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_preds.columns = ['id', 'pred_4']
final_valid_preds.to_csv('valid_pred4.csv', index=False)

submission_data.target = np.mean(np.column_stack(final_test_preds), axis = 1)
submission_data.columns = ['id', 'pred_4']
submission_data.to_csv('test_pred4.csv', index=False)

In [36]:
xtrain.shape

(240000, 24)

In [37]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[24]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1),
])

model.compile(
    optimizer='adam',
    loss='mae',
)

history = model.fit(
    xtrain, ytrain,
    validation_data=(xvalid, yvalid),
    batch_size=256,
    epochs=10,
)

Epoch 1/10
938/938 [==============================] - 4s 3ms/step - loss: 0.9740 - val_loss: 0.6506
Epoch 2/10
938/938 [==============================] - 2s 2ms/step - loss: 0.6084 - val_loss: 0.5886
Epoch 3/10
938/938 [==============================] - 2s 2ms/step - loss: 0.6021 - val_loss: 0.5860
Epoch 4/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5956 - val_loss: 0.6025
Epoch 5/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5980 - val_loss: 0.5987
Epoch 6/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5986 - val_loss: 0.5899
Epoch 7/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5912 - val_loss: 0.5834
Epoch 8/10
938/938 [==============================] - 3s 3ms/step - loss: 0.5913 - val_loss: 0.5857
Epoch 9/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5907 - val_loss: 0.5856
Epoch 10/10
938/938 [==============================] - 2s 2ms/step - loss: 0.5917 - val_loss: 0.5841

In [39]:
xtest.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,4.0,8.0,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,0.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,7.0,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,4.0,3.0,10.0,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,1.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,2.0,5.0,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [40]:
max_t = xtest.max(axis=0)
min_t = xtest.min(axis=0)
xtest = (xtest - min_t) / (max_t - min_t)
xtest.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,0.666667,0.333333,0.333333,0.0,0.571429,0.666667,0.571429,...,0.276211,0.323404,0.420982,0.142413,0.542467,0.199256,0.194339,0.034647,0.358388,0.920114
1,0.0,1.0,0.0,0.666667,0.333333,0.666667,0.0,0.571429,0.333333,0.500000,...,0.037029,0.796818,0.765874,0.762549,0.501285,0.196965,0.497243,0.836727,0.881487,0.719094
2,1.0,0.0,0.0,0.000000,0.333333,0.333333,0.0,0.571429,0.500000,0.714286,...,0.552043,0.624123,0.470532,0.814918,0.412001,0.358340,0.439758,0.325570,0.429733,0.472934
3,1.0,1.0,0.0,0.666667,0.333333,1.000000,0.0,0.571429,0.000000,0.928571,...,0.579602,0.716408,0.715163,0.531388,0.987549,0.315821,0.932869,0.352574,0.445836,0.411042
4,1.0,1.0,0.0,0.666667,0.333333,0.666667,0.0,0.571429,0.333333,0.357143,...,0.071454,0.376900,0.460729,0.247402,0.544524,0.314853,0.595895,0.327160,0.423740,0.992582


In [45]:
preds = model.predict(xtest)
submission_data.target = preds
submission_data.to_csv('submission.csv', index=False)

In [46]:
submission_data.head()

,id,target
0,0,8.023475
1,5,8.340709
2,15,8.238367
3,16,8.204875
4,17,8.216704


In [16]:
def run(trial):
    #optimize in one fold
    fold = 0
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[features]
    xvalid = xvalid[features]

    xtrain[object_cols]= ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.fit_transform(xvalid[object_cols])

    learning_rate = trial.suggest_float("learning_rate", 1e-1, 0.9, log=True)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 0.9)
    max_depth = trial.suggest_int('max_depth', 1, 9)
    subsample = trial.suggest_float('subsample', 0.1, 0.9)
    reg_lambda = trial.suggest_float('reg_lambda', 1, 100.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1, 100.0)
    alpha = trial.suggest_int('alpha', 0, 100)
    n_estimator = trial.suggest_int('n_estimator',200, 1000)

    model = lgb.LGBMRegressor(random_state=0,
                     device='gpu',
                     learning_rate = learning_rate,
                     colsample_bytree = colsample_bytree,
                     max_depth = max_depth,
                     subsample = subsample,
                     reg_lambda = reg_lambda,
                     reg_alpha = reg_alpha
                     )

    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=1000)

[I 2021-08-30 16:50:39,599] A new study created in memory with name: no-name-cd799ee9-c550-4643-b075-0cb89f220ff8
[I 2021-08-30 16:50:44,005] Trial 0 finished with value: 0.7297642703475963 and parameters: {'learning_rate': 0.11199345581901614, 'colsample_bytree': 0.8648270544067962, 'max_depth': 5, 'subsample': 0.21826420196255986, 'reg_lambda': 80.84095373861469, 'reg_alpha': 39.12412728523593, 'alpha': 7, 'n_estimator': 994}. Best is trial 0 with value: 0.7297642703475963.
[I 2021-08-30 16:50:49,640] Trial 1 finished with value: 0.7242266386461075 and parameters: {'learning_rate': 0.20280050334862598, 'colsample_bytree': 0.2609907749582999, 'max_depth': 9, 'subsample': 0.4196533614308905, 'reg_lambda': 28.842439593153003, 'reg_alpha': 20.88244062456174, 'alpha': 0, 'n_estimator': 613}. Best is trial 1 with value: 0.7242266386461075.
[I 2021-08-30 16:50:54,314] Trial 2 finished with value: 0.7235404850038121 and parameters: {'learning_rate': 0.2679765017657509, 'colsample_bytree': 0.

KeyboardInterrupt: 

In [17]:
study.best_params

{'learning_rate': 0.44372268845270024,
 'colsample_bytree': 0.132564808633615,
 'max_depth': 7,
 'subsample': 0.6762612502880989,
 'reg_lambda': 22.893490045935902,
 'reg_alpha': 54.01141982124916,
 'alpha': 15,
 'n_estimator': 413}

In [35]:
final_preds = []
for fold in range(10):
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    xtrain[object_cols]= ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.fit_transform(xvalid[object_cols])
    
    best_params = {'learning_rate': 0.44372268845270024,
                 'colsample_bytree': 0.132564808633615,
                 'max_depth': 7,
                 'subsample': 0.6762612502880989,
                 'reg_lambda': 22.893490045935902,
                 'reg_alpha': 54.01141982124916,
                 'alpha': 15,
                 'n_estimator': 413}
    model = lgb.LGBMRegressor(random_state=0,
                     device='gpu',
                     **best_params
                     )

    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_preds.append(test_preds)
    print(fold, mean_squared_error(yvalid, preds_valid, squared=False))

0 0.7233694971878205
1 0.7204669390661295
2 0.7199945049913765
3 0.7184870825019493
4 0.7270030942495257
5 0.7202854557603239
6 0.7198884246176857


KeyboardInterrupt: 

In [19]:
submission_data.target = np.mean(np.column_stack(final_preds), axis = 1)
submission_data.to_csv('submission.csv', index=False)